# 自动出图

流程：
- 让ai帮我们生成prompt，然后保存到某个txt文件里
- 以这个txt文件为输入，调用已经写好的ComfyUI脚本，生成图片

In [1]:
# 先得安装pyautogen
%pip install pyautogen

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 设置Api入口
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

In [3]:
# 创建完成这个项目需要的agent
llm_config = {
    "config_list": config_list, 
    "seed": 42,
    "functions": [
        {
            "name":"savePrompts",
            "description":"save prompts to a text file",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompts": {
                        "type": "string",
                        "description": "the prompts to be saved",
                    },
                },
                "required": ["message"],
            },
        }
    ],
    }

# 首先，我们需要一个与人类用户交互的proxy agent
user_proxy = autogen.UserProxyAgent(
   name="User_proxy", # 这里的命令规则是只能包含字母(大写和小写)、数字、下划线和连字符，长度在1到64个字符之间。
   system_message="A human admin.",
   code_execution_config={"last_n_messages": 2, "work_dir": "output"},
   human_input_mode="NEVER",
   llm_config=llm_config,
)
# 接下来，我们需要一个把用于中英文翻译的agent
translator = autogen.AssistantAgent(
    name="Translator",
    system_message="Your task is to translate the user's Chinese into English and do nothing else.",
    llm_config=llm_config
)
# 然后，我们需要一个把按用户需求生成绘图prompt的agent
prompts_generator = autogen.AssistantAgent(
    name="Prompts_generator",
    system_message="""
    Your task is to generate some prompts for ai-generated picture. And don't do anything else.
    
    These prompts must meet the following requirements:
    - Include description of attire, background, lighting and shadow.
    - Each sentence should be limited to 50 words.
    
    for example:
        'Wearing a futuristic suit, she stands in front of an abstract mural, neon lights casting bold shadows across her metallic armor, mirroring the vibrant colors of the artwork.'
    """,
    llm_config=llm_config
)

# 我们还需要一个能把prompt文件保存为text的agent
prompts_saver = autogen.UserProxyAgent(
    name="Prompts_saver", 
    system_message="A proxy agent that saves prompts to a text file.",
    code_execution_config={"last_n_messages": 2, "work_dir": "output"},
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    max_consecutive_auto_reply=10,
    llm_config=llm_config
)

def savePrompts(prompts):
    print("got message: ", prompts)
    save_file_path = "output\generated_text.txt"
    try:
        with open(save_file_path, "a") as f:
            f.write(prompts + "\n")
        return f"image prompt file generated ,and saved at {save_file_path}. TERMINATE"
    except Exception as e:
        print(e)
        return f"image prompt file generated failed, error: {e}. TERMINATE"

prompts_saver.register_function(
    function_map={
        "savePrompts": savePrompts,
    }
)

# 然后，我们建一个群把它们放在一起
groupchat = autogen.GroupChat(agents=[user_proxy, translator, prompts_generator, prompts_saver], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [4]:
# 用户输入需求
user_input = "帮我生成一些女网红的写真照片的提示词，然后保存成文件。"

In [5]:
# 执行代码
user_proxy.initiate_chat(manager,message=user_input)

User_proxy (to chat_manager):

帮我生成一些女网红的写真照片的提示词，然后保存成文件。

--------------------------------------------------------------------------------
Translator (to chat_manager):

Help me generate some photo shoot prompt words for female internet celebrities, then save them as a file.

--------------------------------------------------------------------------------
Prompts_generator (to chat_manager):

1. 'Dressed in soft, bohemian style clothing, she is lounging in the warm golden glow of a cozy café, her long, flowing skirts creating fluid shadows on the worn wooden tables.'
2. 'In a chic, modern apartment, lit by the serene light of dawn, she wears a silk bathrobe. Her delicately curled hair casting a soft shadow on the elegant marble floor.'
3. 'Adorned in vibrant workout gear, she is caught midjog in a lush forest, dawn's first rays providing dramatic illumination and casting long, active shadows.'
4. 'Outside a rustic cottage, under the warm evening light, she stands wearing a vintage dr